In [1]:
push!(LOAD_PATH,"../../../ExoJulia/")
push!(LOAD_PATH,".")


using ExoJulia
#using PyPlot
#using Optim
using Transit

# Read in the data

In [4]:
data = readdlm("mystery_planet2.txt");
time = data[:,1];
flux = data[:,2];
mean_flux = flux/median(flux)
err = data[:,3];

# Inform transit of the transits
Transit.set_time(time)
Transit.set_flux(mean_flux)
Transit.set_err(err)

Set Transit.time
Set Transit.flux
Set Transit.err


In [5]:
Transit.guess_params()

[0

-1

.0,0.007,0.014,0.021,0.028]

# What's the data look like?

In [ ]:
scatter(time,mean_flux, color="blue", alpha=0.1)
ylim(0.995,1.005)

# Use Agol Periodogram to estimate the period

In [ ]:
periods = collect(linspace(10.0,20.0,1000))

best_period = agol_periodogram(data, periods)

# Phase fold with the fitted period to see if it works.

In [ ]:
data_fold = copy(data)
data_fold[:,1] = mod(time - time[1], best_period);
data_fold = fastsortrows(data_fold,[1]);

In [ ]:
scatter(data_fold[:,1],data_fold[:,2]/mean(data_fold[:,2]),color="blue")
ylim(0.995,1.005)
#xlim(minimum(data_fold[:,1]),maximum(data_fold[:,1]))
xlim(0.25,1.5)

### We want to calculate the first time that elapses between the first observation and the center of the first transit, $t_{off}$.

In [ ]:
function estimate_guess(time::Array{Float64,1},flux::Array{Float64,1},period::Float64)
    # Estimate 1st transit center
    best_ind = argmin(abs(time-period))
    toff_est_ind = argmin(flux[1:best_ind]) # <- this index ~ 1st transit center
    """
    #=
    cadence = 5
    avg_flux = [ mean(b[i:min(i+cadence-1,end)]) for i=1:cadence:length(b) ]
    
    # Estimate transit duration
    mf = median(flux)
    iter = 1
    T_right_ind = 1
    T_left_ind = 1
    
    # Find right edge of transit
    while abs(1.0 - flux[toff_est_ind+iter]/mf) > 0.001 && iter < length(flux)
        T_right_ind = toff_est_ind + iter
        iter = iter + 1
    end
    
    # Find left edge of transit
    while abs(1.0 - flux[toff_est_ind-iter]/mf) > 0.001 && iter > 0
        T_left_ind = toff_est_ind - iter
        iter = iter - 1
    end
    
    # Better estimate for transit center ind
    toff_est_ind = convert(Int,round((T_left_ind + T_right_ind)/2.0))
    
    
    """
    # Use this to estimate depth assuming i ~ 90 degrees
    df_guess = 1.0 - flux[toff_est_ind]/mean(flux)
    
    return time[toff_est_ind], df_guess
end

In [ ]:
best_toff, best_df = estimate_guess(time,mean_flux,best_period)

In [ ]:
scatter(time,mean_flux, color="blue", alpha=0.1)
scatter(best_toff, 1.0, alpha=1.0, color="k", label="First Transit Center")
ylim(0.995,1.005)
xlim(0.0, 3.0)
legend(loc=0)

# Try fitting this model with optim

In [ ]:
best_optim = optimum.minimum

# Fit to Seager 2003 formalization

## Run with `Optim`

In [ ]:
# params [ dF, tT, tF, P, tE ]
params = [best_df, 0.5, 0.2, best_period, best_toff]

optimum = optimize(transit_loglike_observables_optim, params, iterations=5000, method=:gradient_descent)

In [ ]:
best_optim = optimum.minimum

In [ ]:
plot_results_full(best_optim)

In [ ]:
plot_results_folded(best_optim)